# OllaBridge × Google Colab — Production Demo (4 Scenarios)

**Updated:** 2026-01-06  
This notebook shows four end‑to‑end setups for running LLM inference where the GPU lives **(Colab GPU, your local GPU, or a registered node in the cloud)** while keeping a single OpenAI‑compatible API surface.

We’ll use **OllaBridge Cloud** at:

- **Cloud base:** `https://cloud.ollabridge.com`

---

## Before you start

### What is what?
- **Gateway / Control Plane (OllaBridge):** the OpenAI-compatible API you call (`/v1/...`), handles auth + routing.
- **Node / Worker (ollabridge-node):** a machine that provides compute (usually runs Ollama + models).
- **OllaBridge Cloud :** a hosted control plane + secure relay for nodes behind NAT (no port forwarding).

### Security defaults used in this notebook
- If you don’t configure `API_KEYS`, OllaBridge generates a **per-run secret key** and prints it.
- **By default it does NOT write keys to `.env`.**
- If you *explicitly* want persistence for dev only: `ollabridge start --write-env`

> **Production guidance:** Use environment variables or a secret manager for API keys; avoid committing `.env` files.


## Architecture diagrams

### 1) High-level routing

```mermaid
flowchart LR
  Client[Client (Colab / Local)] -->|OpenAI-compatible HTTP| GW[OllaBridge Gateway]
  GW -->|routes requests| Node[Worker Node(s)]
  Node -->|Ollama / model inference| GPU[(GPU/CPU)]
```

### 2) Cloud relay (no port forwarding)

```mermaid
sequenceDiagram
  participant C as Client (Colab)
  participant Cloud as cloud.ollabridge.com
  participant Dev as Your Device/Node
  C->>Cloud: HTTPS request (OpenAI /v1)
  Cloud->>Dev: WebSocket relay (req)
  Dev->>Cloud: WebSocket response (res/delta/done)
  Cloud->>C: HTTPS response (stream or JSON)
```


---
# ☁️ Scenario 3: OllaBridge Cloud Relay (Local → Cloud → Colab)

**Goal:** Securely access your local machine (GPU) from Colab **without port forwarding**, using **OllaBridge Cloud** as a relay.

**Cloud endpoint:** `https://cloud.ollabridge.com`

## How it works
1. Your **local computer** runs Ollama + your model and connects **outbound** to OllaBridge Cloud over WebSocket (`ollabridge-node connect`).
2. This notebook (Colab) sends OpenAI-compatible requests to **https://cloud.ollabridge.com/v1**.
3. OllaBridge Cloud relays the request to your device and returns the response (optionally streaming).

```mermaid
flowchart LR
  Colab[Google Colab] -->|HTTPS /v1| Cloud[cloud.ollabridge.com]
  Cloud -->|WebSocket relay| Local[Your local node (GPU)]
  Local --> Ollama[Ollama + model]
```

## On your local computer (one-time registration + connect)
Run these commands on your local machine:

```bash
pip install -U ollabridge
ollama pull deepseek-r1  # ensure model exists locally

# Register the device with OllaBridge Cloud (may show a pairing URL/code)
ollabridge-node register --cloud https://cloud.ollabridge.com

# Keep the node connected (relay channel)
ollabridge-node connect --cloud https://cloud.ollabridge.com
```

> After the node is connected, use your **Cloud API key** (`CLOUD_API_KEY`) from the OllaBridge Cloud dashboard in this notebook.


In [ ]:
#@title Variables (edit these)
import os

MODEL_ID = os.environ.get("MODEL_ID", "deepseek-r1")

# Scenario 2: Colab -> your local gateway (paste values when you have them)
LOCAL_GATEWAY_BASE_URL = os.environ.get("LOCAL_GATEWAY_BASE_URL", "")  # e.g. https://xxxx.tunnel.../v1
LOCAL_API_KEY = os.environ.get("LOCAL_API_KEY", "")

# Scenarios 3/4: OllaBridge Cloud
CLOUD_BASE_URL = os.environ.get("CLOUD_BASE_URL", "https://cloud.ollabridge.com/v1")
CLOUD_API_KEY = os.environ.get("CLOUD_API_KEY", "")  # paste from Cloud dashboard or your gateway banner

print("MODEL_ID:", MODEL_ID)
print("CLOUD_BASE_URL:", CLOUD_BASE_URL)


### Helper utilities

These helpers keep the notebook readable:
- `health_check(base_url)`
- `list_models(base_url, api_key)`
- `chat(base_url, api_key, model, message)`
- `chat_stream(base_url, api_key, model, message)` (best-effort streaming)


In [ ]:
import json
import time
import requests
from typing import Optional, Iterable

def _auth_headers(api_key: str) -> dict:
    return {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

def health_check(base_url: str, timeout: int = 5) -> bool:
    if not base_url:
        print("❌ base_url is empty.")
        return False
    # normalize: base_url might be .../v1
    health_url = base_url.replace("/v1", "") + "/health"
    try:
        r = requests.get(health_url, timeout=timeout)
        ok = (r.status_code == 200)
        print(("✅" if ok else "❌"), "GET", health_url, "->", r.status_code)
        if not ok:
            print(r.text[:400])
        return ok
    except Exception as e:
        print("❌ health_check error:", type(e).__name__, str(e)[:200])
        return False

def list_models(base_url: str, api_key: str, timeout: int = 15) -> list[str]:
    url = base_url.rstrip("/") + "/models"
    r = requests.get(url, headers=_auth_headers(api_key), timeout=timeout)
    r.raise_for_status()
    data = r.json()
    return [m.get("id") for m in data.get("data", []) if m.get("id")]

def chat(base_url: str, api_key: str, model: str, message: str, timeout: int = 90) -> str:
    url = base_url.rstrip("/") + "/chat/completions"
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": message}],
        "temperature": 0.3,
    }
    r = requests.post(url, headers=_auth_headers(api_key), json=payload, timeout=timeout)
    r.raise_for_status()
    j = r.json()
    return j["choices"][0]["message"]["content"]

def chat_stream(base_url: str, api_key: str, model: str, message: str, timeout: int = 90) -> Iterable[str]:
    """Best-effort streaming for OpenAI-compatible SSE-style responses.

    If your gateway doesn't support streaming, it may return a normal JSON response.
    """
    url = base_url.rstrip("/") + "/chat/completions"
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": message}],
        "temperature": 0.3,
        "stream": True,
    }
    with requests.post(url, headers=_auth_headers(api_key), json=payload, stream=True, timeout=timeout) as r:
        r.raise_for_status()
        # Some implementations stream as lines; some return JSON once.
        content_type = r.headers.get("Content-Type", "")
        if "application/json" in content_type:
            j = r.json()
            yield j["choices"][0]["message"]["content"]
            return

        for raw in r.iter_lines(decode_unicode=True):
            if not raw:
                continue
            line = raw.strip()
            if line.startswith("data:"):
                line = line[len("data:"):].strip()
            if line == "[DONE]":
                break
            try:
                j = json.loads(line)
                delta = j["choices"][0].get("delta", {}).get("content")
                if delta:
                    yield delta
            except Exception:
                # Not JSON; ignore
                continue


# Scenario 1 — Run OllaBridge *inside* Google Colab (use Colab GPU)

### Goal
Run everything in Colab:
- Ollama (model runtime)
- OllaBridge Gateway (OpenAI-compatible API)
- Inference happens on **Colab GPU**

### Notes (Colab specifics)
- Colab does not run systemd; we start `ollama serve` in the background.
- The first model pull can take time depending on the model size.


## 1.1 Install Ollama + OllaBridge (Colab)

Run this once per runtime.


In [ ]:
!python -m pip -q install -U ollabridge openai requests
!curl -fsSL https://ollama.com/install.sh | sh


## 1.2 Start Ollama server (background)

We start `ollama serve` in the background and wait until it answers.


In [ ]:
import subprocess, shlex, sys, re

# Start ollama serve in background (idempotent-ish)
_ = subprocess.Popen(
    ["bash", "-lc", "nohup ollama serve > /tmp/ollama.log 2>&1 &"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

# Wait for Ollama
for i in range(30):
    try:
        r = requests.get("http://localhost:11434/api/tags", timeout=2)
        if r.status_code == 200:
            print("✅ Ollama is up")
            break
    except Exception:
        pass
    time.sleep(1)
else:
    print("❌ Ollama did not start. Check logs:")
    !tail -n 50 /tmp/ollama.log


## 1.3 Pull a model

Choose a model that fits your Colab GPU quota.


In [ ]:
# Pull the model (example: deepseek-r1)
!ollama pull {MODEL_ID}


## 1.4 Start OllaBridge gateway (Colab)

This exposes the OpenAI-compatible API at:
- `http://localhost:11435/v1`

OllaBridge will print an API key in the banner. Copy it into `COLAB_API_KEY` below.


In [ ]:
# Start the gateway in the background so the notebook continues
_ = subprocess.Popen(
    ["bash", "-lc", "nohup ollabridge start --host 0.0.0.0 --port 11435 --log-level info > /tmp/ollabridge.log 2>&1 &"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

# Show last lines to grab the key
!tail -n 60 /tmp/ollabridge.log


In [ ]:
#@title Paste the API key printed by OllaBridge (Scenario 1)
COLAB_GATEWAY_BASE_URL = "http://localhost:11435/v1"
COLAB_API_KEY = ""  # paste from banner, e.g. sk-ollabridge-...

print("Base URL:", COLAB_GATEWAY_BASE_URL)


In [ ]:
# Smoke test: health + chat
assert health_check(COLAB_GATEWAY_BASE_URL), "Gateway is not healthy yet."

print("Available models (first 10):")
try:
    print(list_models(COLAB_GATEWAY_BASE_URL, COLAB_API_KEY)[:10])
except Exception as e:
    print("Models listing failed (auth key wrong?):", type(e).__name__, str(e)[:200])

print("\nChat result:")
print(chat(COLAB_GATEWAY_BASE_URL, COLAB_API_KEY, MODEL_ID, "Say hello from Colab GPU in one sentence."))


# Scenario 2 — Use your *local* GPU from Google Colab

### Goal
Run the gateway on your computer (using your GPU), and call it from Colab.

### How it works
1. On your computer: start OllaBridge (which uses your local Ollama + GPU).
2. Expose it to the internet safely:
   - Quick dev: `ollabridge start --share`
   - Production: VPN / reverse proxy / **OllaBridge Cloud** (Scenario 3)

Then Colab calls the **public base URL**.

> If you started locally with `--share`, your base URL will look like `https://.../v1`.


## 2.1 On your computer (outside Colab)

Run:

```bash
ollabridge start --share
```

Copy from the banner:
- Public URL (ends with `/v1`)
- API key

Paste them below in this notebook (`LOCAL_GATEWAY_BASE_URL`, `LOCAL_API_KEY`).


In [ ]:
# Scenario 2: health + chat
if not LOCAL_GATEWAY_BASE_URL or not LOCAL_API_KEY:
    print("ℹ️ Paste LOCAL_GATEWAY_BASE_URL and LOCAL_API_KEY in the Variables cell above.")
else:
    ok = health_check(LOCAL_GATEWAY_BASE_URL)
    if ok:
        print("Chat result (runs on your local GPU):")
        print(chat(LOCAL_GATEWAY_BASE_URL, LOCAL_API_KEY, MODEL_ID, "Confirm this response is generated on my local GPU."))


# Scenario 3 — Register your local computer in OllaBridge Cloud and call it from Colab

### Goal
No tunnels. No port forwarding.
Your local computer (GPU) connects **outbound** to the cloud using WebSockets.
Colab sends requests to the cloud, and the cloud relays them to your computer.

**Cloud:** `https://cloud.ollabridge.com`

---

## 3.1 On your local computer (outside Colab)

1) Install and run the cloud-connected node (example commands; use your exact CLI if it differs):

```bash
pip install -U ollabridge
# register device (you may be prompted to complete a pairing step)
ollabridge-node register --cloud https://cloud.ollabridge.com
# connect and stay connected
ollabridge-node connect --cloud https://cloud.ollabridge.com
```

2) Ensure the node has access to Ollama + the model:
```bash
ollama pull deepseek-r1
```

> If your cloud uses a pairing code (like TV login), follow the printed URL/code once during registration.

---

## 3.2 From Colab: call the cloud base URL

Paste your cloud API key into `CLOUD_API_KEY` above (from your Cloud dashboard or gateway settings).


In [ ]:
# Scenario 3: health + chat via cloud
if not CLOUD_API_KEY:
    print("ℹ️ Paste CLOUD_API_KEY in the Variables cell above.")
else:
    ok = health_check(CLOUD_BASE_URL)
    if ok:
        print("Chat result (cloud relays to your registered local GPU node):")
        print(chat(CLOUD_BASE_URL, CLOUD_API_KEY, MODEL_ID, "Hello! Reply with one sentence to confirm relay through OllaBridge cloud."))

        print("\nStreaming demo (best-effort):")
        out = []
        for chunk in chat_stream(CLOUD_BASE_URL, CLOUD_API_KEY, MODEL_ID, "Stream 10 short words separated by spaces."):
            out.append(chunk)
            sys.stdout.write(chunk)
            sys.stdout.flush()
        print("\n\nDone.")


---
# 🖥️ Scenario 4: Register Google Colab as a Cloud Node (Laptop → Cloud → Colab GPU)

**Goal:** Turn this Colab runtime into a **worker node** registered in **OllaBridge Cloud**, so you can use **Colab GPU** from your local computer.

**Cloud endpoint:** `https://cloud.ollabridge.com`

## How it works
- Colab connects outbound to the Cloud (no inbound ports).
- Your laptop (or a local gateway) calls the Cloud API (`https://cloud.ollabridge.com/v1`).
- Cloud relays requests to this Colab node.

```mermaid
flowchart LR
  Laptop[Your laptop / client] -->|HTTPS /v1| Cloud[cloud.ollabridge.com]
  Cloud -->|WebSocket relay| Colab[This Colab node (GPU)]
  Colab --> Ollama[Ollama + model]
```

## Important note
Colab runtimes are **ephemeral**. If the runtime disconnects, the node goes offline and must reconnect/register again.


In [ ]:
# Install (if not already)
!python -m pip -q install -U ollabridge


In [ ]:
# Register this Colab runtime as a node (you may be prompted for pairing)
# NOTE: If your cloud requires browser-based pairing, follow the printed instructions.
!ollabridge-node register --cloud https://cloud.ollabridge.com


In [ ]:
# Connect and keep alive (run this cell and leave it running if needed)
# In Colab, long-running processes can be started in the background as well.
import subprocess, time

_ = subprocess.Popen(
    ["bash", "-lc", "nohup ollabridge-node connect --cloud https://cloud.ollabridge.com > /tmp/ollabridge_node.log 2>&1 &"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
print("✅ Node connect started. Tail logs:")
!tail -n 80 /tmp/ollabridge_node.log


## 4.2 On your local computer: route requests to the Colab node through the cloud

Depending on your product mode, you will do one of the following:

### Option A — Call cloud directly (recommended for demos)
Set your client:
- `base_url = https://cloud.ollabridge.com/v1`
- `api_key = <your CLOUD_API_KEY>`

The cloud will route to the node you select/own (often via device selection in dashboard).

### Option B — Run a local gateway that routes via cloud (if your CLI supports it)
Example (your CLI may differ):
```bash
ollabridge start --cloud https://cloud.ollabridge.com
```

Then your local apps call:
- `http://localhost:11435/v1`

---

### Minimal local client example (Python)
```python
from openai import OpenAI

client = OpenAI(
  base_url="https://cloud.ollabridge.com/v1",
  api_key="YOUR_CLOUD_KEY",
)

resp = client.chat.completions.create(
  model="deepseek-r1",
  messages=[{"role":"user","content":"Run this on Colab GPU"}],
)
print(resp.choices[0].message.content)
```


# Troubleshooting

### 401 Unauthorized
- Ensure your client uses the **same** API key the gateway/cloud expects.
- For local gateway: the key is printed at startup and is set via `API_KEYS`.
- For cloud: use the API key from your cloud account/dashboard.

### Model not found
- Ensure the model exists on the worker machine:
  - `ollama pull <model>`
- Ensure you request the correct model id (`MODEL_ID` in this notebook).

### Cloud relay connected but requests don’t route
- Confirm your node is **connected** and **owned/authorized** for your cloud API key.
- Check device selection/routing rules in the cloud dashboard (if applicable).

---

# Production checklist (quick)
- Use env vars or secret manager for `API_KEYS` / cloud secrets
- Use HTTPS everywhere
- Restrict CORS origins
- Rate limit pairing/login endpoints (cloud)
- Rotate keys regularly



## Using Colab GPU from your laptop (after Scenario 4 is connected)

Once this Colab node is **Connected** in OllaBridge Cloud, you can use it from your laptop in two common ways:

### A) Call OllaBridge Cloud directly (simplest)
- Base URL: `https://cloud.ollabridge.com/v1`
- API key: your `CLOUD_API_KEY`

```python
from openai import OpenAI

client = OpenAI(base_url="https://cloud.ollabridge.com/v1", api_key="YOUR_CLOUD_API_KEY")
resp = client.chat.completions.create(
    model="deepseek-r1",
    messages=[{"role":"user","content":"Run this on Colab GPU via the cloud relay."}],
)
print(resp.choices[0].message.content)
```

### B) Optional: run a local gateway and route to cloud (if supported)
Some deployments support a local gateway that routes to Cloud:

```bash
ollabridge start --cloud https://cloud.ollabridge.com
```

Then call:
- `http://localhost:11435/v1`

> If your current CLI doesn’t support `--cloud`, use option A.
